In [2]:
import numpy as np
import pandas as pd 
import pickle
from glob import glob
import os

In [3]:
# Template stuff
imagenet_root_path = '/bigstor/zsarwar/Imagenet_2012'
imagenet_subsets_path = '/bigstor/zsarwar/Imagenet_2012_subsets'
template_path = os.path.join(imagenet_subsets_path, "Dogs_vs_Wolves_metadata.pkl")
df_template = pd.read_pickle(template_path)
df_template = pd.DataFrame.from_dict(df_template, orient='index')


dataset_path = "/bigstor/common_data/food_101/food-101/images/*"
df_path = "/bigstor/common_data/food_101/DF/"
all_folders = glob(dataset_path)
#all_folders = [fold for fold in all_folders if ".txt" not in fold]
class_mapping = "/bigstor/common_data/food_101/food-101/meta/classes.txt"
label_mapping = "/bigstor/common_data/food_101/food-101/meta/labels.txt"

train_split = "/bigstor/common_data/food_101/food-101/meta/train.txt"
val_split = "/bigstor/common_data/food_101/food-101/meta/test.txt"

In [4]:
# Process label to class mapping
with open(class_mapping, 'r') as i_file:
    all_labels = i_file.readlines()
all_labels = [cl.replace("\n", "") for cl in all_labels]

with open(label_mapping, 'r') as i_file:
    all_classes = i_file.readlines()
all_classes = [cl.replace("\n", "") for cl in all_classes]



In [5]:
labels_classes = {all_labels[i]: all_classes[i] for i in range(len(all_labels))}
labels_idx = {all_labels[i]: i for i in range(len(all_labels))}

In [6]:
cols = df_template.columns
metadata = []
for idx, folder in enumerate(all_folders):
    label = folder.split("/")[-1]
    all_images = glob(folder + "/*")
    for j in range(len(all_images)):
        t_dict = {}
        t_dict[cols[0]] = None
        t_dict[cols[1]] = labels_classes[label]
        t_dict[cols[2]] = labels_idx[label]
        t_dict[cols[3]] = 'food101'
        t_dict[cols[4]] = 'food101'
        t_dict[cols[5]] = None
        t_dict[cols[6]] = all_images[j]
        t_dict['index'] = '/'.join(all_images[j].split("/")[-2:])
        metadata.append(t_dict)        



In [7]:
df_food = pd.DataFrame.from_dict(metadata)
df_food = df_food.set_index('index')
df_food.index.name = None

In [25]:
with open(train_split, "r") as i_file:
    train_images = i_file.readlines()
    train_images = [img.replace("\n", "") + ".jpg" for img in train_images]

with open(val_split, "r") as i_file:
    val_images = i_file.readlines()
    val_images = [img.replace("\n", "") + ".jpg" for img in val_images]

In [31]:
df_train =df_food[df_food.index.isin(train_images)]

In [32]:
df_val = df_food[df_food.index.isin(val_images)]

In [33]:
df_val

,image_url,class,label,data_type,dataset,query,img_path
french_onion_soup/2993508.jpg,None,French onion soup,41,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...
french_onion_soup/1184744.jpg,None,French onion soup,41,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...
french_onion_soup/798877.jpg,None,French onion soup,41,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...
french_onion_soup/3808030.jpg,None,French onion soup,41,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...
french_onion_soup/1959316.jpg,None,French onion soup,41,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...
...,...,...,...,...,...,...,...
bread_pudding/562153.jpg,None,Bread pudding,8,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...
bread_pudding/2443597.jpg,None,Bread pudding,8,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...
bread_pudding/3294452.jpg,None,Bread pudding,8,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...
bread_pudding/255220.jpg,None,Bread pudding,8,food101,food101,None,/bigstor/common_data/food_101/food-101/images/...


In [34]:
out_val_path = os.path.join(df_path, "df_food101_val.pkl")
df_val.to_pickle(out_val_path)
out_train_path = os.path.join(df_path, "df_food101_train.pkl")
df_train.to_pickle(out_train_path)

In [35]:
out_train_path

'/bigstor/common_data/food_101/DF/df_food101_train.pkl'